# Expectation-maximization algorithm

We will derive and implement formulas for Gaussian Mixture Model — one of the most commonly used methods for performing soft clustering of the data. 

Place samples.npz file in the same folder as em_anton.ipynb 

### Possible display problems

If some of the formulas are not displayed correctly, you can view the file [here](https://nbviewer.jupyter.org/github/antonsavostianov/Bayes-in-ML/blob/master/EM/em_anton.ipynb) via nbviewer.

In [ ]:
import numpy as np
from numpy.linalg import slogdet, det, solve
import matplotlib.pyplot as plt
import time
from sklearn.datasets import load_digits

%matplotlib inline

from scipy.special import softmax

from numpy.random import default_rng
# from IPython.display import clear_output

## Implementing EM for GMM

For debugging, we will use samples from a Gaussian mixture model with unknown mean, variance, and priors.

In [ ]:
samples = np.load('samples.npz')
X = samples['data']
pi0 = samples['pi0']
mu0 = samples['mu0']
sigma0 = samples['sigma0']
plt.scatter(X[:, 0], X[:, 1], c='grey', s=30)
plt.axis('equal')
plt.show()

In [ ]:
type(samples)
print(samples.files)

In [ ]:
print("X.shape",X.shape)
print("X[0]",X[0])
print("sigma0.shape",sigma0.shape)

### Mathematical Model

We aim to seek for the probability density of the data as a mixture of simpler distributions (in this particular case - Gaussians). Each Gaussian component determines its cluster (class). The number of Gaussians (= the number of clusters) to use is our choice (in our example it is $3$). Assuming that samples are independent the density reads

$$
p(X,T|\theta)=\prod_{i=1}^n p(x_i,t_i|\theta)=\prod_{i=1}^n p(x_i|t_i,\theta)p(t_i|\theta)=\prod_{i=1}^n f_{\!\mathcal{N}}(x_i| \mu_{t_i}, \Sigma_{t_i})\pi_{t_i},
$$

where $X=\{x_i\}_{i=1}^n$ - sample points ($x_i\in{\mathbb R}^d$), $T=\{t_i\}^n_{i=1}$ - the class of the corresponding point ($t_i\in\{1,2,...,C\}$, $t$ for tag),

$f_{\!\mathcal{N}}(x| \mu, \Sigma)= \frac{1}{(2\pi)^{d/2}\sqrt{|det \Sigma|}}exp\Big(-\frac{1}{2}(x-\mu)^t\Sigma^{-1}(x-\mu)\Big)$ - density of a normal distribution with mean $\mu$ and covariance matrix $\Sigma$,

$\theta = \{\pi_{c},\mu_{c},\Sigma_{c}\}_{c=1}^C$ - parameters of our model: $\pi_{c}$ - probability of belonging to the class $c$, $(\mu_{c},\Sigma_{c})$ - mean and covariance of the Gaussian corresponding to the class $c$.

The fact that $p(t_i|\theta)$ does not depend on $\theta$ ($p(t_i|\theta)=\pi_{t_i}$) is our model assumption.

Notice that if we knew the tag $t_i$ for each point we could separately consider the points of each class $c$, and consequently estimate the parameters
$\mu_c,\ \Sigma_c$ (via sample mean and covariance matrix) and find $\pi_c = \frac{\#\{t_i=c\}}{n}$. The main difficulty is that we do not know the tags! 


## Setting optimization problem
So we consider $p(X|\theta)$ and according to the maxlikelihood approach we should maximize it with respect to $\theta$
$$
\log\, p(X|\theta)\to \max\limits_{\theta}.
$$
We first rewrite $\log p(X|\theta)$ using an auxiliary distribution $q(t_i)$ over $\{1,...,C\}$:

$$
q(t_i):\{1,...,C\}\to[0,1],\ q(t_i)(c):=q(t_i=c), \sum_{c=1}^Cq(t_i=c)=1. 
$$

We have

\begin{multline}
\log(p(X\mid\theta))=\sum_{i=1}^N\log p(x_i\mid \theta) = \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log p(x_i\mid \theta)= \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(\frac{p(x_i\mid \theta) p(x_i, t_i = c\mid \theta)}{p(x_i, t_i = c\mid \theta)}\right) = \\
\sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(\frac{p(x_i, t_i = c\mid \theta)}{p(t_i = c\mid x_i, \theta)}\right) = \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(\frac{q(t_i=c)}{p(t_i = c\mid x_i, \theta)}\right) + \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(\frac{p(x_i, t_i = c\mid \theta)}{q(t_i=c)}\right)=\\
\sum_{i=1}^N \mathcal{KL}(q(t_i)\mid\mid p(t_i|x_i,\theta)) + \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(\frac{p(x_i, t_i = c\mid \theta)}{q(t_i=c)}\right):=\mathcal{KL}(q(T)|| p(T|X,\theta))+\mathcal L(\theta,q).
\end{multline}

That is 

$$
\log(p(X\mid\theta)) = \mathcal{KL}(q(T)|| p(T|X,\theta))+\mathcal L(\theta,q).
$$

Since $\mathcal{KL}(q(T)|| p(T|X,\theta))\geq 0$ (can be checked) we see that $\log(p(X\mid\theta))\geq \mathcal L(\theta,q)$. It is known that $\mathcal L(\theta,q)$ is a variational lower bound for $\log p(X|\theta)$. Therefore instead of the original problem we consider the new one

$$
\mathcal L(\theta,q)\to \max\limits_{\theta,q}.
$$

To maximize $\mathcal L(\theta,q)$ we do coordinatewise updates:

\begin{equation*}
q_k=argmax_q\mathcal L(\theta_{k-1},q),\ \theta_k=argmax_\theta \mathcal L(\theta,q_k). 
\end{equation*}

Let us consider each step seaparately:

<b>E-step</b>: due to $\log(p(X\mid\theta)) = \mathcal{KL}(q(T)|| p(T|X,\theta))+\mathcal L(\theta,q)$ we have<br>
$\mathcal{L}(\theta, q) \to \max\limits_{q} \Leftrightarrow  \mathcal{KL} (q(T) \,||\, p(T|X, \theta)) \to \min \limits_{q} \Rightarrow q(T) = p(T|X, \theta)$<br>
<b>M-step</b>:<br> 
$\mathcal{L}(\theta, q) \to \max\limits_{\theta} \Leftrightarrow \mathbb{E}_{q(T)}\log p(X,T | \theta) \to \max\limits_{\theta}$

### E-step
In this step we need to estimate the posterior distribution over the latent variables with fixed values of parameters: $q(t_i) = p(t_i \mid x_i, \theta)$. We assume that $t_i$ equals to the cluster index of the true component of the $x_i$ object. To do so we need to compute $\gamma_{ic} = p(t_i = c \mid x_i, \theta)$. Note that $\sum\limits_{c=1}^C\gamma_{ic}=1$.


\begin{align}
\gamma_{ic} = p(t_i = c \mid x_i, \theta) = \frac{p(x_i,\theta \mid t_i= c)\,p(t_i=c)}{p(x_i,\theta)} = \frac{p(x_i\mid t_i= c, \theta )\,p(\theta\mid t_i=c)\,p(t_i=c)}{p(x_i,\theta)}= |\theta\ and\ t_i - independent|=\\
\frac{p(x_i\mid t_i= c, \theta )\,p(\theta)\,p(t_i=c)}{\sum\limits_{c'=1}^C p(x_i,t_i=c', \theta)} = \frac{p(x_i\mid t_i= c, \theta )\,p(\theta)\,p(t_i=c)}{\sum\limits_{c'=1}^C p(x_i \mid t_i=c', \theta)p(t_i=c'\mid \theta)p(\theta)} = \frac{p(x_i\mid t_i= c, \theta )\pi_c}{\sum\limits_{c'=1}^C p(x_i \mid t_i=c', \theta)\pi_{c'}} 
\end{align}

Now we compute
\begin{align}
p(x_i\mid t_i= c, \theta )\pi_c = & f_{\mathcal{N}}(x_i\mid \mu_c,\Sigma_c)\pi_c = \frac{1}{((2\pi)^d|\det\Sigma_c|)^{1/2}}\exp\left(-\frac{1}{2}\Sigma_c^{-1}(x_i-\mu_c).(x_i-\mu_c)\right)\pi_c = \\
& \exp\left(-\frac{1}{2}\Sigma_c^{-1}(x_i-\mu_c).(x_i-\mu_c) + \log(\pi_c) - \frac{d}{2}\log(2\pi) - \frac{1}{2}\log|\det\Sigma_c| \right).
\end{align}


<b>usefull functions: </b> <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.slogdet.html">```slogdet```</a> and <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html#numpy.linalg.det">```det```</a>

In [ ]:
def E_step(X, pi, mu, sigma):
    """
    Performs E-step on GMM model
    Each input is numpy array:
    X: (N x d), data points
    pi: (C), mixture component weights 
    mu: (C x d), mixture component means
    sigma: (C x d x d), mixture component covariance matrices
    
    Returns:
    gamma: (N x C), probabilities of clusters for objects
    """
    N = X.shape[0] # number of objects
    C = pi.shape[0] # number of clusters
    d = mu.shape[1] # dimension of each object
    gamma = np.zeros((N, C)) # distribution q(T)
    
    log_sigma = np.linalg.slogdet(sigma)[1]
    
    logits = np.zeros((C,N))
    
    for c in range(C):
        yc = np.linalg.solve(sigma[c],(X - mu[c]).T).T
        logits[c] = -0.5 * np.sum(yc*(X-mu[c]),axis = 1) + np.log(pi[c])-np.log(2*np.pi) - 0.5*log_sigma[c]
    logits = logits.T

    gamma = softmax(logits, axis = 1)
    
    return gamma

In [ ]:
gamma = E_step(X, pi0, mu0, sigma0)
print(gamma[0:10])

### M-step

In M-step we need to maximize $\mathbb{E}_{q(T)}\log p(X,T | \theta)$ with respect to $\theta$. In our model this means that we need to find optimal values of $\pi$, $\mu$, $\Sigma$. We start by deriving formulas for $\mu$ as it is the easiest part. Then move on to $\Sigma$. Finaly, to compute $\pi$, we need <a href="https://www3.nd.edu/~jstiver/FIN360/Constrained%20Optimization.pdf">Lagrange Multipliers technique</a> to satisfy constraint $\sum\limits_{i=1}^{n}\pi_i = 1$.

<br>
<b>Note:</b> We need to compute derivatives of scalars with respect to matrices. The necessary information can be found on<a href="https://en.wikipedia.org/wiki/Matrix_calculus"> wiki article</a> about it . Main formulas of matrix derivatives can be found in <a href="http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf">Chapter 2 of The Matrix Cookbook</a>. One of the useful formulas is $\frac{\partial}{\partial A}\log |A| = A^{-T}$.

Maximization of $\mathbb{E}_{q(T)}\log p(X,T | \theta) = \sum\limits_{i=1}^N\sum\limits_{c=1}^C q(t_i=c)\log \left(\frac{p(x_i, t_i = c\mid \theta)}{q(t_i=c)}\right)$ with respect to $\theta$ for $q(t_i)$ fixed is equivalent to maximization of the expression

\begin{align}
\mathcal{M} = \sum_{i=1}^N\sum_{c=1}^C q(t_i=c)\log \left(p(x_i, t_i = c\mid \theta)\right) = 
\sum_{i=1}^N\sum_{c=1}^C \gamma_{ic}\log \left(p(x_i\mid t_i = c, \theta)\pi_c\right) = 
\sum_{i=1}^N\sum_{c=1}^C \gamma_{ic}\log \left(f_{\!\mathcal{N}}(x_i\mid \mu_c, \Sigma_c)\pi_c\right) = \\ 
\sum_{i=1}^N\sum_{c=1}^C \gamma_{ic}\left(-\frac{1}{2}\Sigma_c^{-1}(x_i-\mu_c).(x_i-\mu_c) + \log(\pi_c) - \frac{d}{2}\log(2\pi) - \frac{1}{2}\log|\det\Sigma_c| \right).
\end{align}

### Step 1: $\mu_c$.

Using that for $A^t=A$ we have $\nabla\frac{1}{2}(Ax.x) = Ax$, one finds

\begin{equation*}
\partial_{\mu_c}\mathcal{M} = \sum_{i=1}^N\gamma_{ic}\left(\Sigma_c^{-1}(x_i - \mu_c)\right) = \Sigma_c^{-1}\left(\sum_{i=1}^N\gamma_{ic}(x_i-\mu_c)\right)=0, 
\end{equation*}
that is 
\begin{equation*}
\mu_c = \frac{\sum_{i=1}^N\gamma_{ic}\,x_i}{\sum_{i=1}^N\gamma_{ic}}.
\end{equation*}

### Step 2: $\Sigma_c$.

We notice that:

\begin{equation*}
\mathcal{M} = \sum_{i=1}^N\sum_{c=1}^C \gamma_{ic}\left(-\frac{1}{2}\Sigma_c^{-1}(x_i-\mu_c).(x_i-\mu_c) + \log(\pi_c) - \frac{n}{2}\log(2\pi) + \frac{1}{2}\log|\det\Sigma_c^{-1}| \right),
\end{equation*}
so it is more convenient to find $\partial_{\Sigma_c^{-1}}\mathcal{M}$.

Using that 
\begin{equation*}
\partial_A \left(\frac{1}{2}Ax.x\right) = \frac{1}{2}x\otimes x, \qquad \partial_A\ln |\det A| = (A^{-1})^t,
\end{equation*}
we find (taking into account that $\Sigma_c^t=\Sigma_c$)
\begin{equation*}
\partial_{\Sigma_c^{-1}}\mathcal{M} = \sum_{i=1}^N\gamma_{ic}\left(-\frac{1}{2}(x_i-\mu_c)\otimes (x_i-\mu_c) + \frac{1}{2}\Sigma_c\right)=0,
\end{equation*}
that implies
\begin{equation*}
\Sigma_c = \frac{\sum_{i=1}^N\gamma_{ic}(x_i-\mu_c)\otimes (x_i-\mu_c)}{\sum_{i=1}^N\gamma_{ic}}.
\end{equation*}

### Step 3: $\pi_c$.

We want to solve the problem
\begin{equation}
\label{pic1}
\begin{cases}
\tag{$\pi_c:1$}
\mathcal{M} \to \max\limits_{\{\pi_c\}},\\
\sum\limits_{c=1}^C\pi_c=1,\ and\ \pi_c\geq 0,\ c\in C.
\end{cases}
\end{equation}
Setting
\begin{equation*}
\gamma_c = \sum_{i=1}^N \gamma_{ic},\ c \in \overline{1,C}.
\end{equation*}

Obviously, the above optimization problem is equivalent to the following one

\begin{equation}
\label{pic2}
\begin{cases}
\tag{$\pi_c:2$}
\sum\limits_{c=1}^C\gamma_c\log\pi_c \to \max\limits_{\{\pi_c\}},\\
\sum\limits_{c=1}^C\pi_c=1,\ and\ \pi_c\geq 0,\ c\in C,
\end{cases}
\end{equation}

or equivalently

\begin{equation}
\label{pic3}
\begin{cases}
\tag{$\pi_c:3$}
-\sum\limits_{c=1}^C\gamma_c\log\pi_c \to \min\limits_{\{\pi_c\}},\\
\sum\limits_{c=1}^C\pi_c-1=0,\ and\ -\pi_c\leq 0,\ c\in C.
\end{cases}
\end{equation}

Let us write down the Lagrange function:
\begin{equation*}
La = -\sum\limits_{c=1}^C\gamma_c\log\pi_c +\alpha\left(\sum\limits_{c=1}^C\pi_c-1\right)+\sum\limits_{c=1}^C\beta_c(-\pi_c).
\end{equation*}

The neccesary conditions of the extremum reads
\begin{equation*}
\begin{cases}
\partial_{\pi_c}La = -\frac{\gamma_c}{\pi_c}+\alpha-\beta_c=0,\ c \in \overline{1,C};\\
\beta_c\pi_c=0,\ \beta_c\geq 0,\ c \in \overline{1,C};\\
\sum\limits_{c=1}^C\pi_c=1.
\end{cases}
\end{equation*}
From this system we observe:
\begin{align}
&-\gamma_c+\alpha\pi_c-\beta_c\pi_c=0 \quad \Rightarrow \quad -\gamma_c+\alpha\pi_c = 0 \quad \Rightarrow\\
& \pi_c = \frac{\gamma_c}{\alpha}\quad \Rightarrow \quad 1=\frac{\sum^C_{c=1}\gamma_c}{\alpha}= \frac{C}{\alpha} \quad \Rightarrow \quad \alpha = C \quad \Rightarrow\\
&\pi_c = \frac{\gamma_c}{C},\ c \in \overline{1,C}.
\end{align}


In [ ]:
def M_step(X, gamma):
    """
    Performs M-step on GMM model
    Each input is numpy array:
    X: (N x d), data points
    gamma: (N x C), distribution q(T)  
    
    Returns:
    pi: (C)
    mu: (C x d)
    sigma: (C x d x d)
    """
    N = X.shape[0] # number of objects
    C = gamma.shape[1] # number of clusters
    d = X.shape[1] # dimension of each object
    
    gamma_c = np.sum(gamma, axis = 0)
    
    mu = gamma.T@X
    mu = ((mu.T)/gamma_c).T
    
    pi = gamma_c/C
    
    sigma = np.zeros((C,d,d))
    
    for c in range(C):
        
        # v.shape - N x d
        v = X - mu[c]
        
        sigma[c] = np.array([np.sum(gamma[:,c]*v[:,m]*v[:,n]) for m in range(d) for n in range(d)]).reshape((d,d))
        sigma[c] = sigma[c]/gamma_c[c]
        
    return pi, mu, sigma

In [ ]:
# check
sigma1 = np.array([np.identity(2) for c in range(3)])
gamma = E_step(X, pi0, mu0, sigma1)
pi, mu, sigma = M_step(X, gamma)
print("det(sigma)",det(sigma))

### Loss function

Finally, we need some function to track convergence. We will use variational lower bound $\mathcal L = \mathcal{L}(\theta,q)$ for this purpose. We will stop our EM iterations when $\mathcal{L}$ will saturate. Usually, we need only about 10-20 iterations to converge. It is also useful to check that this function never decreases during training. If it does, there is a bug in the code.

\begin{align}
\mathcal{L} = \sum_{i=1}^{N} \sum_{c=1}^{C} q(t_i =c) (\log \pi_c + \log f_{\!\mathcal{N}}(x_i \mid \mu_c, \Sigma_c)) - \sum_{i=1}^{N} \sum_{c=1}^{K} q(t_i =c) \log q(t_i =c)=\\
\sum_{i=1}^N\sum_{c=1}^C \gamma_{ic}\left(-\frac{1}{2}\Sigma_c^{-1}(x_i-\mu_c).(x_i-\mu_c) + \log(\pi_c) - \frac{n}{2}\log(2\pi) - \frac{1}{2}\log|\det\Sigma_c| \right) - \sum_{i=1}^N\sum_{c=1}^C\gamma_{ic}\log(\gamma_{ic}).
\end{align}

In [ ]:
def compute_vlb(X, pi, mu, sigma, gamma):
    """
    Each input is numpy array:
    X: (N x d), data points
    gamma: (N x C), distribution q(T)  
    pi: (C)
    mu: (C x d)
    sigma: (C x d x d)
    
    Returns value of variational lower bound
    """
    N = X.shape[0] # number of objects
    C = gamma.shape[1] # number of clusters
    d = X.shape[1] # dimension of each object
    
    log_sigma = np.linalg.slogdet(sigma)[1]
    
    logits = np.zeros((C,N))
    
    for c in range(C):
        yc = np.linalg.solve(sigma[c],(X - mu[c]).T).T
        logits[c] = -0.5 * np.sum(yc*(X-mu[c]),axis = 1) + np.log(pi[c])-np.log(2*np.pi) - 0.5*log_sigma[c]
    logits = logits.T
    
    loss = np.sum(gamma*logits - gamma*np.log(gamma))

    return loss

In [ ]:
pi, mu, sigma = pi0, mu0, sigma0
gamma = E_step(X, pi, mu, sigma)
pi, mu, sigma = M_step(X, gamma)
loss = compute_vlb(X, pi, mu, sigma, gamma)
print(loss)

### Bringing it all together

Now that we have E step, M step and VLB, we can implement the training loop. We will initialize values of $\pi$, $\mu$ and $\Sigma$ to some random numbers, train until $\mathcal{L}$ stops changing, and return the resulting points. We also know that the EM algorithm converges to local optima. To find a better local optima, we will restart the algorithm multiple times from different (random) starting positions. Each training trial should stop either when maximum number of iterations is reached or when relative improvement is smaller than given tolerance ($|\frac{\mathcal{L}_i-\mathcal{L}_{i-1}}{\mathcal{L}_{i-1}}| \le \text{rtol}$).

Remember, that initial (random) values of $\pi$ that you generate must be non-negative and sum up to 1. We use $\Sigma=I$ as initialization.

Sometimes we get numerical errors because of component collapsing. The easiest way to deal with this problems is to restart the procedure.

In [ ]:
def train_EM(X, C, rtol=1e-3, max_iter=100, restarts=10):
    '''
    Starts with random initialization *restarts* times
    Runs optimization until saturation with *rtol* reached
    or *max_iter* iterations were made.
    
    X: (N, d), data points
    C: int, number of clusters
    '''
    N = X.shape[0] # number of objects
    d = X.shape[1] # dimension of each object
    best_loss = None
    best_pi = None
    best_mu = None
    best_sigma = None    
    
    
    rng = default_rng()

    for trial in range(restarts):
        try:
            i, rtol_cur = 0, 1
            
            pi_old = softmax(rng.random(C))
            mu_old = 8*rng.random((C,d)) # numbers in the square (0,8)x(0,8)

            sigma_old = np.array([np.identity(d) for c in range(C)]) 
            
            loss_history = []
            
            while (i<max_iter) and (rtol_cur > rtol):
                i=i+1
                gamma = E_step(X, pi_old, mu_old, sigma_old)
                pi_new, mu_new, sigma_new = M_step(X,gamma)
                loss = compute_vlb(X, pi_new, mu_new, sigma_new, gamma)
                loss_history.append(loss)
                pi_old, mu_old, sigma_old = pi_new, mu_new, sigma_new

                if i==1:
                    rtol_cur = 1
                    loss_old = loss
                if i>1:
                    rtol_cur = (loss-loss_old)/(loss_old)
                    loss_old = loss
                
                if (best_loss == None) or (loss > best_loss):
                    # condition loss>best_loss since loss = L(thetea,q) is maximized
                    best_loss = loss
                    best_pi, best_mu, best_sigma = pi_new, mu_new, sigma_new
            
            #clear_output()
            print("trial",trial,": loss",loss)
            plt.plot(np.array(loss_history))
            plt.show()
            plt.clf()
          
        except np.linalg.LinAlgError:
            print("trial",trial, ";", "iteration",i)
            print("Singular matrix: components collapsed")
            print("det(sigma_old)", det(sigma_old))
            print()
            pass

    return best_loss, best_pi, best_mu, best_sigma

In [ ]:
best_loss, best_pi, best_mu, best_sigma = train_EM(X, 3)

Let's plot the clusters to see it. We will assign a cluster label as the most probable cluster index. This can be found using a matrix $\gamma$ computed on last E-step. 

In [ ]:
gamma = E_step(X, best_pi, best_mu, best_sigma)
labels = gamma.argmax(axis=1)
colors = np.array([(31, 119, 180), (255, 127, 14), (44, 160, 44)]) / 255.
plt.scatter(X[:, 0], X[:, 1], c=colors[labels], s=30)
plt.axis('equal')
plt.show()